# I Cambiamenti Climatici

In [1]:
#magic
%load_ext autoreload
%autoreload 2

In [2]:
#import librerie
import pandas as pd
import numpy as np
import os
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import requests
import datetime
import jsonstat
from IPython.display import IFrame

from grafici import trend_co2,trend_temp,perc_rid,perc_rid_map,perc_spesa

# Key and Secret
config = {} 
config_path = os.path.join(os.path.abspath('/opt/app/data/config')) 
config_name = r'config_od.py' 
config_file = os.path.join(config_path,config_name) 
exec(open(config_file).read(),config)

# Plotly settings
plotly_user=config['PLOTLY_USER']
plotly_key=config['PLOTLY_KEY']
plotly.tools.set_credentials_file(username=plotly_user, api_key=plotly_key)

dir_out = os.path.join(os.path.abspath(''),'output')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', None)

## Mondo

#### G0. Ultimo valore Anidride Carbonica (CO2)

In [3]:
def last_value():
    #Leggo il file messo a disposizione da Mauna Loa
    url="ftp://aftp.cmdl.noaa.gov/products/trends/co2/co2_mm_mlo.txt"
    columns = ['Anno','Mese','Data Decimale','Media CO2','Interpolato','Trend','#days']
    co2_mese = pd.read_csv(url,comment='#',sep = '\s+', names=columns)
    # Seleziono Ultimo Valore
    last_co2 = co2_mese.iloc[-1:]['Media CO2'].values[0]
    last_anno = co2_mese.iloc[-1:]['Anno'].values[0]
    last_mese = co2_mese.iloc[-1:]['Mese'].values[0]
    print('Ultimo Valore Medio: {} ppm\n Anno : {}\n Mese : {}\n'.format(last_co2, last_anno, last_mese))
    
    # Seleziono Primo Valore
    first_co2 = co2_mese.iloc[1:]['Media CO2'].values[0]
    first_anno = co2_mese.iloc[1:]['Anno'].values[0]
    first_mese = co2_mese.iloc[1:]['Mese'].values[0]
    print('Primo Valore Medio: {} ppm\n Anno : {}\n Mese : {}'.format(first_co2, first_anno, first_mese))

last_value()

Ultimo Valore Medio: 411.75 ppm
 Anno : 2019
 Mese : 2

Primo Valore Medio: 317.45 ppm
 Anno : 1958
 Mese : 4


#### G1. Trend annuale di concentrazione Anidrire Carbonica (CO2)

> Unità di misura: in ppm

> Fonte: Osservatorio Mauna Loa, Hawaii (NOAA-ESRL)

In [4]:
#Leggo il file messo a disposizione da Mauna Loa
url="ftp://aftp.cmdl.noaa.gov/products/trends/co2/co2_annmean_mlo.txt"
columns = ['Anno','Media CO2','unc']
co2_anno = pd.read_csv(url,comment='#',sep = '\s+', names=columns)
del co2_anno['unc']

In [ ]:
# Grafico
#fig_co2 = trend_co2(co2_anno)
#py.iplot(fig_co2, filename='co2-trend')

In [6]:
# Grafico
IFrame('//plot.ly/~andrea.cesarini85/56.embed', width=450, height=300)

#### G2. Paesi per CO2 prodotta in migliaia di tonnellate (kton)

> Unità di misura: in kton

> Fonte: World Bank Data

In [151]:
IFrame('https://data.worldbank.org/share/widget?indicators=EN.ATM.CO2E.KT&view=map', width=450, height=300)

#### G3. Incremento temperatura terrestre

> Unità di misura: gradi C°

> Fonte: NASA

In [5]:
#Leggo il file messo a disposizione dal NOAA
url="https://data.giss.nasa.gov/gistemp/tabledata_v3/ZonAnn.Ts+dSST.txt"
temp = pd.read_csv(url, sep = '\n', header=-1)
global_temp = pd.read_csv(url, skiprows=8, nrows=21,sep = '\s+', header=0)
global_temp1 = pd.read_csv(url, skiprows=32, nrows=20,sep = '\s+', header=0)
global_temp2 = pd.read_csv(url, skiprows=55, nrows=20,sep = '\s+', header=0)
global_temp3 = pd.read_csv(url, skiprows=78, nrows=20,sep = '\s+', header=0)
global_temp4 = pd.read_csv(url, skiprows=101, nrows=20,sep = '\s+', header=0)
global_temp5 = pd.read_csv(url, skiprows=124, nrows=20,sep = '\s+', header=0)
global_temp6 = pd.read_csv(url, skiprows=147, nrows=16,sep = '\s+', header=0)
global_temp = global_temp.append(global_temp1).append(global_temp2).append(global_temp3).append(global_temp4).append(global_temp5).append(global_temp6)

# Filtro solo le colonne d'interesse e setto anno indice
global_temp = global_temp.filter(items=['Year', 'Glob'])
#global_temp = global_temp.set_index(['Year'])

# Normalizzo la temperatura
global_temp['Glob_C'] = global_temp['Glob']*0.01 
del global_temp['Glob']

In [9]:
# Grafico
#fig_temp = trend_temp(global_temp)
#py.iplot(fig_temp, filename='temp-trend')

In [10]:
# Grafico
IFrame('//plot.ly/~andrea.cesarini85/58.embed', width=450, height=300)

## Europa

> Dataset 1 - EU Institutions - https://data.europa.eu/euodp/en/data 

The European Union Open Data Portal (EU ODP) provides access to an expanding range of data from the European Union (EU) institutions and other EU bodies

#### GDP

In [6]:
# From EU Institution Eurostat get GDP, Country and People
# Uso query builder https://ec.europa.eu/eurostat/web/json-and-unicode-web-services/getting-started/query-builder
# CP_MEUR -> Current prices, million eur
# B1GQ -> Gross domestic product (GDP) at market prices
url_1 = 'http://ec.europa.eu/eurostat/wdds/rest/data/v1.1/json/en/nama_10_gdp?na_item=B1GQ&precision=1&unit=CP_MEUR'
file_name_1 = r'GDP.json'
j = jsonstat.download(url_1, file_name_1)
collection_1 = jsonstat.from_file('data/GDP.json')
gdp = collection_1.dataset('nama_10_gdp')
# Dimensioni Geo e Time
df_gdp = gdp.to_table(content='id',rtype=pd.DataFrame)
# Filtro solo 2016
df_gdp=df_gdp[(df_gdp['time']=='2016')]

In [7]:
df_gdp.head(2)

,unit,na_item,geo,time,Value
41,CP_MEUR,B1GQ,AL,2016,10740.0
85,CP_MEUR,B1GQ,AT,2016,356237.6


#### Number Of Abitant

In [8]:
# Uso query builder https://ec.europa.eu/eurostat/web/json-and-unicode-web-services/getting-started/query-builder
# POP_NC -> Totale national population
# THS_PER -> thousand person
url_2 = 'http://ec.europa.eu/eurostat/wdds/rest/data/v1.1/json/en/nama_10_pe?na_item=POP_NC&precision=1&unit=THS_PER'
file_name_2 = r'POP.json'
j = jsonstat.download(url_2, file_name_2)
collection_2 = jsonstat.from_file('data/POP.json')
pop = collection_2.dataset('nama_10_pe')
# Dimensioni Geo e Time
df_pop = pop.to_table(content='id',rtype=pd.DataFrame)
# Filtro solo 2016
df_pop=df_pop[(df_pop['time']=='2016')]

#### G4. Emissione Gas Serra

In [9]:
# Uso query builder https://ec.europa.eu/eurostat/web/json-and-unicode-web-services/getting-started/query-builder
# GHG -> Greenhouse gas
# CRF1-6X4_MEMO -> All sector
# THS_T -> ktons
url_3 = 'http://ec.europa.eu/eurostat/wdds/rest/data/v1.1/json/en/env_air_gge?airpol=GHG&precision=1&airemsect=CRF1-6X4_MEMO&unit=THS_T'
file_name_3 = r'GHG.json'
j = jsonstat.download(url_3, file_name_3)
collection_3 = jsonstat.from_file('data/GHG.json')
ghg = collection_3.dataset('env_air_gge')
# Dimensioni Geo e Time
df_ghg = ghg.to_table(content='id',rtype=pd.DataFrame)

In [10]:
df_gdp.rename(columns={'Value': 'GDP'},inplace=True)
df_gdp.drop(columns=['na_item', 'unit'], inplace=True)
df_pop.rename(columns={'Value': 'Habitant'},inplace=True)
df_pop.drop(columns=['na_item', 'unit'], inplace=True)
df_ghg.rename(columns={'Value': 'GHG'},inplace=True)
df_ghg.drop(columns=['airpol', 'unit','airemsect'], inplace=True)

**Le emissioni dell’UE sono ridotte del 23% tra il 1990 e il 2016**

In [11]:
#1990
ghg_eu_1990 = df_ghg[(df_ghg['time']=='1990') & (df_ghg['geo'] == 'EU28')]['GHG'].values[0]
ghg_eu_2016 = df_ghg[(df_ghg['time']=='2016') & (df_ghg['geo'] == 'EU28')]['GHG'].values[0]
riduzione = ((ghg_eu_1990-ghg_eu_2016)/ghg_eu_1990)*100
riduzione

23.995900313202963

**Riduzione delle emissioni per Paese, GDP e Numero Abitanti**

In [12]:
ghg_1990 = df_ghg[(df_ghg['time']=='1990')]
ghg_2016 = df_ghg[(df_ghg['time']=='2016')]
ghg_diff = ghg_2016.set_index(['geo']).join(ghg_1990.set_index(['geo']),rsuffix='1990').reset_index()

In [13]:
ghg_diff['perc_rid']=((ghg_diff['GHG1990']-ghg_diff['GHG'])/ghg_diff['GHG1990'])*100

In [14]:
df_all = ghg_diff.set_index(['geo','time']).join(df_pop.set_index(['geo','time'])).join(df_gdp.set_index(['geo','time'])).reset_index()

In [15]:
df_all.head(2)

,geo,time,GHG,time1990,GHG1990,perc_rid,Habitant,GDP
0,AT,2016,79672.64,1990,78690.05,-1.248684,8739.81,356237.6
1,BE,2016,117727.11,1990,146654.02,19.724594,11295.00,424660.3


In [16]:
#geo -> Paese
#time -> Anno
#GDP -> Gross Domestic Product in milioni di eur (ovvero il PIL)
#Habitant -> Numero abitanti per migliaia di persone
#GHG -> Greenhouse emission in Ktons del 2016
#perc_rid -> % di riduzione delle emissioni dal 1990 al 2016
df_all.sort_values('perc_rid',ascending=True).head(7)

,geo,time,GHG,time1990,GHG1990,perc_rid,Habitant,GDP
33,TR,2016,496067.36,1990,210714.73,-135.421302,NaN,780224.9
4,CY,2016,8773.06,1990,5591.21,-56.908075,851.56,18490.2
18,IS,2016,4669.34,1990,3634.19,-28.483651,335.40,18735.0
28,PT,2016,67621.06,1990,59825.04,-13.031366,10325.50,186480.5
10,ES,2016,324706.55,1990,287656.37,-12.880014,46449.87,1118743.0
17,IE,2016,61545.82,1990,55490.29,-10.912774,4748.99,273238.2
26,NO,2016,53242.51,1990,51696.96,-2.989634,5236.00,335747.5


In [19]:
# Setto valori negativi a ZERO  Turchia e Cipro i pessimi quelli che hanno aumentato le emissioni
df_all['perc_rid'][df_all['perc_rid'] < 0] = 0

In [20]:
df_code = pd.read_csv('https://datahub.io/core/country-list/r/data.csv')

In [21]:
df_all = df_all.set_index(['geo']).join(df_code.set_index(['Code'])).reset_index()

In [22]:
df_all.head(2)

,geo,time,GHG,time1990,GHG1990,perc_rid,Habitant,GDP,Name
0,AT,2016,79672.64,1990,78690.05,0.000000,8739.81,356237.6,Austria
1,BE,2016,117727.11,1990,146654.02,19.724594,11295.00,424660.3,Belgium


In [26]:
# Grafico
#fig_rid = perc_rid(df_all)
#py.iplot(fig_rid, filename='perc_rid')

In [54]:
# Grafico
IFrame('//plot.ly/~andrea.cesarini85/60.embed', width=750, height=600)

In [28]:
# Grafico
#fig_map = perc_rid_map(df_all)
#py.iplot(fig_map, filename = 'perc_rid_map')

In [29]:
# Grafico
IFrame('//plot.ly/~andrea.cesarini85/62.embed', width=450, height=300)

#### GX. Spesa per Cambiamenti Climatici (rispetto spesa totale)

In [31]:
df_all.head(2)

,geo,time,GHG,time1990,GHG1990,perc_rid,Habitant,GDP,Name
0,AT,2016,79672.64,1990,78690.05,0.000000,8739.81,356237.6,Austria
1,BE,2016,117727.11,1990,146654.02,19.724594,11295.00,424660.3,Belgium


In [39]:
# Contribution to the international 100bn USD commitment on climate related expending (source: DG CLIMA, EIONET)
# Spesa per Cambiamento Climatico in Milioni di Euro
url_4 = 'http://ec.europa.eu/eurostat/wdds/rest/data/v1.1/json/en/sdg_13_50?unit=MIO_EUR&precision=1'
file_name_4 = r'SpendingClimateChange.json'
j = jsonstat.download(url_4, file_name_4)
collection_4 = jsonstat.from_file('data/SpendingClimateChange.json')
spclima = collection_4.dataset('sdg_13_50')
# Dimensioni Geo e Time
df_spesa_clima = spclima.to_table(content='id',rtype=pd.DataFrame)

In [49]:
df_spesa_clima[df_spesa_clima['geo']=='EU28'] #mld spesa

,unit,geo,time,Value
36,MIO_EUR,EU28,2014,11718.43
37,MIO_EUR,EU28,2015,13813.88
38,MIO_EUR,EU28,2016,15501.44


In [50]:
# Filtro solo 2016
df_spesa_clima=df_spesa_clima[(df_spesa_clima['time']=='2016')]
df_spesa_clima.rename(columns={'Value': 'Spesa Clima'},inplace=True)
df_spesa_clima.drop(columns=['unit','time'], inplace=True)

In [25]:
# Spesa complessiva in milioni di euro
url_5 = 'http://ec.europa.eu/eurostat/wdds/rest/data/v1.1/json/en/gov_10a_main?na_item=TE&precision=1&unit=MIO_EUR&sector=S13'
file_name_5 = r'Spending.json'
j = jsonstat.download(url_5, file_name_5)
collection_5 = jsonstat.from_file('data/Spending.json')
spesa = collection_5.dataset('gov_10a_main')
# Dimensioni Geo e Time
df_spesa = spesa.to_table(content='id',rtype=pd.DataFrame)
# Filtro solo 2016
df_spesa=df_spesa[(df_spesa['time']=='2016')]

df_spesa.rename(columns={'Value': 'Spesa'},inplace=True)
df_spesa.drop(columns=['unit','sector','na_item','time'], inplace=True)

In [26]:
df_spesa[df_spesa['geo']=='IT']

,geo,Spesa
1317,IT,828882.0


In [27]:
df_all = df_all.set_index(['geo']).join(df_spesa.set_index(['geo'])).join(df_spesa_clima.set_index(['geo'])).reset_index()

In [28]:
df_all['perc_spesa'] = df_all['Spesa Clima']/df_all['Spesa']*100 # Spesa per il clima in percentuale sulla spesa totale

In [29]:
df_all['perc_spesa'].fillna(0, inplace=True)

In [95]:
# Grafico
#fig_spesa = perc_spesa(df_all)
#py.iplot(fig_spesa, filename = 'perc_spesa')

In [96]:
# Grafico
IFrame('//plot.ly/~andrea.cesarini85/64.embed', width=750, height=600)

11    15.50144
Name: Spesa Clima, dtype: float64

#### GX. Matrix Correlation cause dei gas serra